# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, string, random

from torch.utils.data import Dataset, DataLoader

from torchinfo import summary

# Конфигурация

In [2]:
RANDOM_SEED = 42
BATCH_SIZE = 1

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
torch.cuda.random.manual_seed_all(RANDOM_SEED)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
device

device(type='cuda')

# Загрузка данных

In [4]:
data = pd.read_csv('../data/all_reviews.csv')

In [5]:
data.head()

,store_name,store_type,store_address,store_district,text,stars,date
0,Дикси,Супермаркет,"Суворовский проспект, 36",Смольнинское,"""Дикси"", как ""Дикси"" Постоянно нужно следить, ...",3.0,2023-03-22
1,Дикси,Супермаркет,"Суворовский проспект, 36",Смольнинское,"""Дикси"", как ""Дикси"" Постоянно нужно следить, ...",3.0,2023-03-22
2,Пятерочка,Супермаркет,"Лиговский проспект, 107",Владимирский,"""Пятёрочка"" выручает, вернее и не скажешь:)",5.0,2023-05-10
3,Дикси,Супермаркет,"Большая Московская, 5",Владимирский,",где суповые наборы,где овощные смеси, ?в холо...",2.0,2022-07-15
4,Дикси,Супермаркет,"Чайковского, 55",Литейный,"«3» только после того, как стал свидетелем диа...",3.0,2022-05-21


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   store_name      5576 non-null   object 
 1   store_type      5576 non-null   object 
 2   store_address   5576 non-null   object 
 3   store_district  5576 non-null   object 
 4   text            5576 non-null   object 
 5   stars           5574 non-null   float64
 6   date            5576 non-null   object 
dtypes: float64(1), object(6)
memory usage: 305.1+ KB


# Загрузка модели

In [7]:
tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
model = AutoModelForSequenceClassification.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")

In [8]:
for p in model.parameters():
    p.requires_grad = False

In [9]:
summary(model)

Layer (type:depth-idx)                                       Param #
XLMRobertaForSequenceClassification                          --
├─XLMRobertaModel: 1-1                                       --
│    └─XLMRobertaEmbeddings: 2-1                             --
│    │    └─Embedding: 3-1                                   (192,001,536)
│    │    └─Embedding: 3-2                                   (394,752)
│    │    └─Embedding: 3-3                                   (768)
│    │    └─LayerNorm: 3-4                                   (1,536)
│    │    └─Dropout: 3-5                                     --
│    └─XLMRobertaEncoder: 2-2                                --
│    │    └─ModuleList: 3-6                                  (85,054,464)
├─XLMRobertaClassificationHead: 1-2                          --
│    └─Linear: 2-3                                           (590,592)
│    └─Dropout: 2-4                                          --
│    └─Linear: 2-5                                      

In [10]:
model.to(device).eval();

# Обработка данных

In [11]:
list_target = ["текст про персонал магазина",
               "текст про цены товаров",
               "текст про ассортимент товаров",
               "текст про качество товаров",
               "текст про чистоту магазина",
               "текст про расположение магазина"]

In [12]:
dict_target = dict(zip(range(len(list_target)),list_target))
dict_target

{0: 'текст про персонал магазина',
 1: 'текст про цены товаров',
 2: 'текст про ассортимент товаров',
 3: 'текст про качество товаров',
 4: 'текст про чистоту магазина',
 5: 'текст про расположение магазина'}

In [13]:
class ReviewDataset(Dataset):

    def __init__(self, data, list_target, tokenizer, device):
        self.data = data['text'].values
        self.tokenizer = tokenizer
        self.length = data.shape[0]
        self.device = device
        self.list_target = list_target
        self.s_option = self.preprocess_target(self.list_target)

    def preprocess_target(self, list_target):
        list_ABC = [x for x in string.ascii_uppercase]
        list_label = [x + '.' if x[-1] != '.' else x for x in list_target]
        list_label_pad = list_label + [tokenizer.pad_token]* (20 - len(list_label))
        s_option = ' '.join(['('+list_ABC[i]+') '+list_label_pad[i] for i in range(len(list_label_pad))])
        return s_option
    
    def __len__(self):
        return self.length

    def __getitem__(self, index): 
        return self.s_option + ' ' + self.tokenizer.sep_token + ' ' + self.data[index]
    
    def collate_fn(self, batch):
        input_ids = tokenizer(text=batch,
                              truncation=True,
                              padding=True,
                              max_length=512,
                              return_tensors='pt')['input_ids']
        return input_ids

In [14]:
dataset = ReviewDataset(data=data,
                        list_target=list_target,
                        tokenizer=tokenizer,
                        device=device)
loader = DataLoader(dataset=dataset,
                    collate_fn=dataset.collate_fn,
                    batch_size=BATCH_SIZE)

# Инференс

In [15]:
%%time
test_pred = []

with torch.no_grad():
    for i, input_ids in enumerate(loader): 
        input_ids = input_ids.to(device)
        logits = model(input_ids=input_ids).logits[:,:len(list_target)]
        prediction = torch.argmax(logits, dim=-1)
        test_pred += prediction.tolist()

CPU times: total: 1min 54s
Wall time: 2min 10s


In [16]:
!nvidia-smi

Fri Sep  8 14:09:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0    47W / 115W |   1892MiB /  8192MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
test_answer = list(map(lambda x: dict_target[x], test_pred))

In [18]:
data['Parameter'] = test_answer

In [19]:
data.head()

,store_name,store_type,store_address,store_district,text,stars,date,Parameter
0,Дикси,Супермаркет,"Суворовский проспект, 36",Смольнинское,"""Дикси"", как ""Дикси"" Постоянно нужно следить, ...",3.0,2023-03-22,текст про качество товаров
1,Дикси,Супермаркет,"Суворовский проспект, 36",Смольнинское,"""Дикси"", как ""Дикси"" Постоянно нужно следить, ...",3.0,2023-03-22,текст про качество товаров
2,Пятерочка,Супермаркет,"Лиговский проспект, 107",Владимирский,"""Пятёрочка"" выручает, вернее и не скажешь:)",5.0,2023-05-10,текст про чистоту магазина
3,Дикси,Супермаркет,"Большая Московская, 5",Владимирский,",где суповые наборы,где овощные смеси, ?в холо...",2.0,2022-07-15,текст про персонал магазина
4,Дикси,Супермаркет,"Чайковского, 55",Литейный,"«3» только после того, как стал свидетелем диа...",3.0,2022-05-21,текст про персонал магазина


In [22]:
data.to_csv('../data/all_reviews_parameter.csv',
            index=False)